In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

from keras.models import Sequential
from keras.layers import LSTM, Bidirectional, Dense, Dropout, Conv1D, MaxPooling1D, GlobalAveragePooling1D, BatchNormalization, Activation
from keras import regularizers
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

import numpy as np


from sklearn.decomposition import TruncatedSVD
from scipy.sparse import csr_matrix, hstack

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\LEGION\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\LEGION\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\LEGION\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
# Load the datasets
train_data = pd.read_csv('train.csv')
val_data = pd.read_csv('val.csv')
test_data = pd.read_csv('test.csv')

lengthTestData = len(test_data)

In [3]:
# Combine text data for preprocessing
text = pd.concat([train_data['text'], val_data['text']], ignore_index=True)
text_test = test_data['text']

In [4]:
linguistic_features = train_data.iloc[:,-31:]
val_linguistic_features = val_data.iloc[:,-31:]
test_linguistic_features = test_data.iloc[:,-31:]

In [5]:
linguistic_features

,Words_per_Sentence,Percentage_First_Person_Singular,Percentage_Third_Person,Percentage_Exclusive,Percentage_Negation,Percentage_Causation,Percentage_Sense,Percentage_PositiveEmo,Percentage_NegativeEmo,Percentage_AffectiveTerms,...,Percentage_PastVerb,Percentage_PresentVerb,Percentage_FutureVerb,Percentage_Article,Percentage_Pronoun,Percentage_Conjunction,Lexical_Diversity,Emotiveness,Rate_of_Adjectives_Adverbs,Flesch_Kincaid
0,28.777778,0.000000,1.930502,0.000000,0.000000,2.702703,5.405405,3.474903,3.474903,6.949807,...,3.861004,4.633205,0.772201,5.019305,4.247104,5.405405,0.548263,0.247059,0.081081,14.1
1,32.900000,0.000000,2.735562,1.519757,0.607903,1.519757,4.863222,2.735562,0.607903,3.343465,...,2.127660,2.127660,0.911854,7.294833,5.471125,5.167173,0.595745,0.296610,0.106383,13.5
2,30.416667,0.000000,2.191781,0.821918,0.547945,1.095890,6.027397,3.013699,0.000000,3.013699,...,3.835616,3.013699,1.095890,8.219178,4.931507,3.561644,0.495890,0.246479,0.095890,11.8
3,21.254545,0.085543,4.790419,1.710864,1.197605,2.138580,3.079555,2.053037,1.796407,3.849444,...,2.138580,4.790419,0.427716,6.843456,8.297690,5.560308,0.437981,0.415254,0.125749,9.9
4,24.444444,0.000000,3.181818,1.636364,0.545455,2.181818,6.181818,3.545455,1.727273,5.272727,...,2.818182,5.818182,0.818182,9.000000,5.909091,4.727273,0.440000,0.273171,0.101818,10.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48634,17.466667,1.908397,3.053435,1.145038,0.381679,0.381679,3.053435,3.053435,3.053435,6.106870,...,4.961832,8.015267,0.000000,4.580153,17.557252,3.816794,0.553435,0.392405,0.118321,8.4
48635,23.500000,3.723404,1.063830,0.531915,0.531915,1.063830,1.063830,4.255319,5.851064,10.106383,...,6.382979,6.382979,0.000000,6.382979,9.574468,4.787234,0.505319,0.357143,0.132979,8.4
48636,25.785714,0.000000,2.908587,1.800554,0.415512,0.969529,0.831025,0.969529,2.216066,3.462604,...,1.800554,5.955679,0.831025,7.063712,5.678670,5.401662,0.488920,0.283636,0.108033,12.8
48637,26.809524,0.532860,8.703375,1.598579,0.355240,1.598579,2.131439,2.131439,4.085258,6.216696,...,3.552398,5.506217,0.000000,4.440497,13.499112,5.150977,0.488455,0.333333,0.110124,9.8


In [6]:
val_linguistic_features

,Words_per_Sentence,Percentage_First_Person_Singular,Percentage_Third_Person,Percentage_Exclusive,Percentage_Negation,Percentage_Causation,Percentage_Sense,Percentage_PositiveEmo,Percentage_NegativeEmo,Percentage_AffectiveTerms,...,Percentage_PastVerb,Percentage_PresentVerb,Percentage_FutureVerb,Percentage_Article,Percentage_Pronoun,Percentage_Conjunction,Lexical_Diversity,Emotiveness,Rate_of_Adjectives_Adverbs,Flesch_Kincaid
0,28.866667,0.000000,2.078522,0.461894,0.000000,2.078522,1.154734,0.230947,2.540416,2.771363,...,2.078522,2.078522,0.230947,11.316397,6.004619,3.233256,0.496536,0.368421,0.129330,11.8
1,15.666667,5.319149,5.851064,1.595745,1.063830,0.000000,0.000000,2.659574,0.000000,2.659574,...,1.595745,3.191489,0.000000,5.851064,13.829787,2.659574,0.595745,0.460000,0.122340,7.7
2,24.709677,0.000000,1.958225,2.349869,1.174935,1.044386,6.527415,2.480418,0.783290,3.394256,...,3.916449,4.569191,1.436031,8.746736,4.830287,4.438642,0.437337,0.232283,0.077023,11.0
3,25.076923,1.533742,4.447853,1.380368,0.613497,1.533742,5.674847,1.993865,0.460123,2.453988,...,5.521472,3.527607,0.460123,9.355828,9.969325,5.368098,0.444785,0.312500,0.092025,9.7
4,25.000000,0.000000,2.000000,1.000000,0.000000,2.000000,2.000000,6.000000,1.000000,7.000000,...,3.000000,2.000000,0.000000,11.000000,3.000000,3.000000,0.690000,0.297297,0.110000,11.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6075,56.166667,0.296736,4.154303,1.186944,1.186944,0.593472,5.341246,2.670623,2.077151,4.747774,...,2.373887,5.044510,0.296736,8.011869,7.715134,4.154303,0.554896,0.274336,0.091988,12.0
6076,22.900000,0.000000,2.183406,0.873362,0.436681,0.000000,5.240175,1.310044,3.493450,4.803493,...,3.056769,3.493450,0.000000,6.113537,4.366812,2.620087,0.620087,0.271605,0.096070,10.7
6077,18.095238,1.842105,5.263158,2.105263,1.052632,1.315789,5.789474,3.684211,0.789474,4.736842,...,5.789474,4.736842,0.263158,4.736842,18.947368,5.789474,0.484211,0.362832,0.107895,6.8
6078,16.611354,1.156677,2.970557,2.602524,1.445846,1.393270,2.628812,2.523659,1.919033,4.468980,...,3.548896,5.783386,0.578339,6.887487,10.778128,6.834911,0.289432,0.457865,0.128549,8.2


In [7]:
test_linguistic_features

,Words_per_Sentence,Percentage_First_Person_Singular,Percentage_Third_Person,Percentage_Exclusive,Percentage_Negation,Percentage_Causation,Percentage_Sense,Percentage_PositiveEmo,Percentage_NegativeEmo,Percentage_AffectiveTerms,...,Percentage_PastVerb,Percentage_PresentVerb,Percentage_FutureVerb,Percentage_Article,Percentage_Pronoun,Percentage_Conjunction,Lexical_Diversity,Emotiveness,Rate_of_Adjectives_Adverbs,Flesch_Kincaid
0,25.066667,0.000000,2.393617,1.595745,0.000000,2.393617,7.446809,2.393617,1.861702,4.255319,...,6.648936,3.191489,0.797872,11.170213,5.585106,5.585106,0.486702,0.345133,0.103723,9.9
1,23.333333,0.000000,2.857143,2.857143,2.857143,2.142857,2.857143,2.142857,0.000000,2.142857,...,7.857143,1.428571,2.142857,5.000000,9.285714,1.428571,0.742857,0.137255,0.050000,12.0
2,24.333333,0.000000,1.369863,1.369863,1.369863,0.000000,8.219178,0.000000,0.000000,0.000000,...,4.109589,0.000000,1.369863,6.849315,1.369863,1.369863,0.698630,0.181818,0.054795,7.7
3,27.466667,0.404531,3.559871,0.566343,0.485437,1.779935,3.478964,2.831715,1.699029,4.530744,...,4.126214,2.993528,0.485437,8.656958,6.877023,4.773463,0.411812,0.285360,0.093042,11.6
4,40.666667,0.000000,3.278689,0.819672,0.000000,3.278689,6.557377,2.459016,2.459016,4.918033,...,6.557377,3.278689,0.819672,8.196721,4.918033,2.459016,0.672131,0.472222,0.139344,12.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6076,26.708333,0.468019,4.680187,2.184087,0.936037,1.560062,1.872075,1.560062,2.808112,4.368175,...,3.432137,4.056162,1.092044,4.368175,11.076443,6.240250,0.436817,0.325472,0.107644,10.1
6077,32.700000,0.000000,1.834862,0.305810,0.000000,0.000000,4.587156,0.000000,5.504587,5.504587,...,1.529052,4.892966,0.000000,9.785933,3.975535,4.892966,0.483180,0.405405,0.137615,10.4
6078,25.600000,0.000000,4.427083,0.781250,0.520833,3.125000,2.083333,4.687500,1.562500,6.250000,...,1.822917,5.989583,0.781250,6.510417,5.468750,5.989583,0.476562,0.297101,0.106771,11.2
6079,63.000000,0.000000,5.555556,2.777778,0.000000,0.793651,2.380952,1.190476,1.587302,2.777778,...,2.777778,1.984127,0.793651,8.333333,8.333333,1.984127,0.575397,0.270000,0.107143,26.6


In [8]:
# Define stopwords and lemmatizer
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

In [9]:
# Function to preprocess text
def preprocess_text(text):
    tokens = word_tokenize(text.lower())  # Tokenization and lowercase
    filtered_tokens = [lemmatizer.lemmatize(token) for token in tokens if token not in stop_words]  # Remove stopwords and lemmatize
    return ' '.join(filtered_tokens)

In [10]:
text_preprocessed = text.apply(preprocess_text)

In [11]:
print(text_preprocessed)

0        beirut ( reuters ) - iran military chief met s...
1        hanoi ( reuters ) - top u.s. envoy began two-d...
2        ( reuters ) - four u.s. senator asked senate j...
3        first read morning briefing meet press nbc pol...
4        cairo ( reuters ) - six month egypt election ,...
                               ...                        
54714    lack oversight prof donald trump totally unfit...
54715    tucker carlson responded espn anchor calling p...
54716    getting something nothing rage president profe...
54717    black emanuelle fixed 1976. attila speaking eu...
54718    chaos broke legal american illegal alien clash...
Name: text, Length: 54719, dtype: object


In [12]:
print(text)

0        beirut (reuters) - iran s military chief met w...
1        hanoi (reuters) - a top u.s. envoy began a two...
2        (reuters) - four u.s. senators have asked the ...
3        first read is a morning briefing from meet the...
4        cairo (reuters) - six months before egypt s el...
                               ...                        
54714    this lack of oversight proves that donald trum...
54715    tucker carlson responded to an espn anchor cal...
54716    because getting something for nothing is all t...
54717    black emanuelle fixed all that in 1976. attila...
54718    chaos broke out after legal americans and ille...
Name: text, Length: 54719, dtype: object


In [13]:
# Initialize TF-IDF vectorizer without specifying max_features
tfidf_vectorizer = TfidfVectorizer()

In [14]:
# Fit and transform the preprocessed text data
tfidf_matrix = tfidf_vectorizer.fit_transform(text_preprocessed)

In [15]:
# Get the number of unique tokens
num_unique_tokens = len(tfidf_vectorizer.get_feature_names_out())
print(num_unique_tokens)

169079


In [16]:
# Re-initialize TF-IDF vectorizer with the determined max_features
tfidf_vectorizer = TfidfVectorizer(max_features=num_unique_tokens)

In [17]:
#Fit and transform the text data again with the updated max_features
tfidf_matrix = tfidf_vectorizer.fit_transform(text_preprocessed)

# Convert the TF-IDF matrix to a CSR (Compressed Sparse Row) matrix for efficient row-wise operations
csr_tfidf_matrix = csr_matrix(tfidf_matrix)

# Find the row index with the maximum number of filled values
max_features_row_index = csr_tfidf_matrix.getnnz(axis=1).argmax()

# Get the number of features in the document with the most filled values
max_features = csr_tfidf_matrix[max_features_row_index].count_nonzero()

print(max_features)

svd = TruncatedSVD(n_components=int(max_features*0.3))
tfidf_matrix = svd.fit_transform(tfidf_matrix)

3498


In [18]:
dense_tfidf_matrix = tfidf_matrix[:len(train_data)]
dense_tfidf_with_linguistic = hstack([dense_tfidf_matrix, csr_matrix(linguistic_features)])

dense_val_tfidf_matrix = tfidf_matrix[len(train_data):len(train_data) + len(val_data)]
dense_val_tfidf_with_linguistic = hstack([dense_val_tfidf_matrix, csr_matrix(val_linguistic_features)])

In [19]:
#Define the CNN + Bi-LSTM model
model = Sequential()
model.add(Conv1D(filters=256, kernel_size=2, activation='relu', input_shape=(dense_tfidf_with_linguistic.shape[1], 1)))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=64, kernel_size=4, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Bidirectional(LSTM(64, return_sequences=True)))
model.add(Dropout(0.3))
model.add(Bidirectional(LSTM(32)))
model.add(Dropout(0.3))
model.add(Dense(16))
model.add(Dropout(0.3))
model.add(Dense(1, activation='sigmoid'))

#Define the Bi-LSTM model
model = Sequential()
model.add(Bidirectional(LSTM(128, return_sequences=True), input_shape=(tfidf_matrix.shape[1],1)))
model.add(Dropout(0.3))
model.add(Bidirectional(LSTM(64)))
model.add(Dropout(0.3))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(1, activation='sigmoid'))

In [20]:
# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [21]:
# Assuming train_data['label'] and val_data['label'] are Pandas Series, convert them to arrays
train_labels = train_data['label'].values
val_labels = val_data['label'].values

early_stopping = EarlyStopping(monitor='val_loss', patience=5, verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, verbose=1)

# Reshape the input data
dense_tfidf_with_linguistic = dense_tfidf_with_linguistic.toarray()  # Convert to dense array
dense_val_tfidf_with_linguistic = dense_val_tfidf_with_linguistic.toarray()  # Convert to dense array

# Fit the model using NumPy arrays with callbacks
model.fit(dense_tfidf_with_linguistic, train_labels, 
          epochs=100, batch_size=32, 
          validation_data=(dense_val_tfidf_with_linguistic, val_labels),
          callbacks=[early_stopping, reduce_lr])

Epoch 1/100
1520/1520 [==============================] - 544s 351ms/step - loss: 0.4435 - accuracy: 0.7915 - val_loss: 0.3030 - val_accuracy: 0.8694 - lr: 0.0010
Epoch 2/100
1520/1520 [==============================] - 539s 354ms/step - loss: 0.3048 - accuracy: 0.8723 - val_loss: 0.2719 - val_accuracy: 0.8883 - lr: 0.0010
Epoch 3/100
1520/1520 [==============================] - 537s 353ms/step - loss: 0.2808 - accuracy: 0.8841 - val_loss: 0.2575 - val_accuracy: 0.8931 - lr: 0.0010
Epoch 4/100
1520/1520 [==============================] - 533s 350ms/step - loss: 0.2683 - accuracy: 0.8896 - val_loss: 0.2855 - val_accuracy: 0.8817 - lr: 0.0010
Epoch 5/100
1520/1520 [==============================] - 494s 325ms/step - loss: 0.2594 - accuracy: 0.8951 - val_loss: 0.2491 - val_accuracy: 0.8974 - lr: 0.0010
Epoch 6/100
1520/1520 [==============================] - 526s 346ms/step - loss: 0.2496 - accuracy: 0.8980 - val_loss: 0.2489 - val_accuracy: 0.8934 - lr: 0.0010
Epoch 7/100
1520/1520 [=====

In [22]:
text_test_preprocessed = text_test.apply(preprocess_text)
test_tfidf_matrix = tfidf_vectorizer.transform(text_test_preprocessed)
dense_test_tfidf_matrix = svd.transform(test_tfidf_matrix)
dense_test_tfidf_with_linguistic = hstack([dense_test_tfidf_matrix, csr_matrix(test_linguistic_features)])
test_labels = test_data['label'].values

dense_test_tfidf_with_linguistic = dense_test_tfidf_with_linguistic.toarray()  # Convert to dense array

# Evaluate the model using the test data
test_loss, test_accuracy = model.evaluate(dense_test_tfidf_with_linguistic, test_labels)
print(f'Test accuracy: {test_accuracy}')

191/191 [==============================] - 25s 112ms/step - loss: 0.2307 - accuracy: 0.9117
Test accuracy: 0.9116921424865723


In [23]:
model.save('bi_lstm_tfidf.h5')

In [24]:
test_accuracy * lengthTestData

5543.999918460846